# YouTube Ranking

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

## 첫페이지

In [3]:
# 크롬드라이버로 링크 열기
url = 'https://youtube-rank.com/board/bbs/board.php?bo_table=youtube'
driver = webdriver.Chrome('chromedriver')
driver.get(url)
time.sleep(2)

* step1, 2

In [5]:
# html 소스 받아서 저장
soup = BeautifulSoup(driver.page_source, 'html.parser')
trs = soup.select('tr.aos-init')
len(trs)

100

* step 3

In [35]:
# 랭크
tr = trs[0]
tr.select_one('.rank').get_text().strip()

'1'

In [36]:
# 카테고리
tr.select_one('.category').get_text().strip().replace('[', '').replace(']', '')

'음악/댄스/가수'

In [37]:
# 채널명
tr.select_one('.subject a').get_text().strip()

'BLACKPINK'

In [38]:
# 구독자, 조회수, 비디오갯수
subscriber = tr.select_one('.subscriber_cnt').get_text().strip()
view = tr.select_one('.view_cnt').get_text().strip()
video = tr.select_one('.video_cnt').get_text().strip()
subscriber, view, video

('8380만', '286억1398만', '466개')

In [24]:
# 리스트에 저장
lines = []
for tr in trs:
    rank = tr.select_one('.rank').get_text().strip()
    category = tr.select_one('.category').get_text().strip().replace('[', '').replace(']', '')
    channel = tr.select_one('.subject a').get_text().strip()
    subscriber = tr.select_one('.subscriber_cnt').get_text().strip()
    view = tr.select_one('.view_cnt').get_text().strip()
    video = tr.select_one('.video_cnt').get_text().strip()
    lines.append([rank, category, channel, subscriber, view, video])

In [25]:
# 데이터 프레임으로 변환
df = pd.DataFrame(lines, columns = ['순위', '카테고리', '채널명', '구독자수', '조회수', '비디오수'])

In [26]:
df

,순위,카테고리,채널명,구독자수,조회수,비디오수
0,1,음악/댄스/가수,BLACKPINK,8380만,286억1398만,466개
1,2,음악/댄스/가수,BANGTANTV,7300만,191억4327만,"2,086개"
2,3,음악/댄스/가수,HYBE LABELS,6950만,258억4674만,"1,065개"
3,4,음악/댄스/가수,SMTOWN,3130만,262억1822만,"4,052개"
4,5,키즈/어린이,Boram Tube Vlog [보람튜브 브이로그],2650만,110억5288만,223개
...,...,...,...,...,...,...
95,96,TV/방송,JTBC Drama,469만,50억8639만,"27,000개"
96,98,키즈/어린이,CoCosToy 꼬꼬스토이,467만,28억7649만,656개
97,97,음식/요리/레시피,Cooking tree 쿠킹트리,467만,4억6943만,"1,410개"
98,99,음식/요리/레시피,까니짱 [ G-NI : 蟹ちゃん],465만,13억5018만,719개


In [49]:
# 불필요한 문자들 바꿔주는 함수
def convert_unit(s):
    s = s.replace('만', '0000').replace('억', '').replace('개', '').replace(',', '')
    return f'{int(s):,d}'

In [52]:
# 컬럼에 함수 적용
    # df['구독자수'] = df['구독자수'].apply(convert_unit)
    # df['조회수'] = df['구독자수'].apply(convert_unit)
    # df['비디오수'] = df['구독자수'].apply(convert_unit)
for column in df.columns[3:]:
    df[column] = df[column].apply(convert_unit)
df

,순위,카테고리,채널명,구독자수,조회수,비디오수
0,1,음악/댄스/가수,BLACKPINK,"83,800,000","28,613,980,000",466
1,2,음악/댄스/가수,BANGTANTV,"73,000,000","19,143,270,000","2,086"
2,3,음악/댄스/가수,HYBE LABELS,"69,500,000","25,846,740,000","1,065"
3,4,음악/댄스/가수,SMTOWN,"31,300,000","26,218,220,000","4,052"
4,5,키즈/어린이,Boram Tube Vlog [보람튜브 브이로그],"26,500,000","11,052,880,000",223
...,...,...,...,...,...,...
95,96,TV/방송,JTBC Drama,"4,690,000","5,086,390,000","27,000"
96,98,키즈/어린이,CoCosToy 꼬꼬스토이,"4,670,000","2,876,490,000",656
97,97,음식/요리/레시피,Cooking tree 쿠킹트리,"4,670,000","469,430,000","1,410"
98,99,음식/요리/레시피,까니짱 [ G-NI : 蟹ちゃん],"4,650,000","1,350,180,000",719


## 모든 페이지

In [ ]:
# https://youtube-rank.com/board/bbs/board.php?bo_table=youtube&page=
# tqdm을 사용해 10 페이지
# youtubeRank.csv 저장

In [54]:
from tqdm.notebook import tqdm

In [59]:
lines = []
for i in tqdm(range(1, 11)):
    # 크롬드라이버로 링크 열기
    url = f'https://youtube-rank.com/board/bbs/board.php?bo_table=youtube&page={i}'
    driver = webdriver.Chrome('chromedriver')
    driver.get(url)
    time.sleep(2)
    # html 소스 받아서 저장
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    trs = soup.select('tr.aos-init')
    # 필요한 데이터 불러오기
    for tr in trs:
        rank = tr.select_one('.rank').get_text().strip() # 랭크
        category = tr.select_one('.category').get_text().strip().replace('[', '').replace(']', '') #카테고리
        channel = tr.select_one('.subject a').get_text().strip() # 채널명
        subscriber = tr.select_one('.subscriber_cnt').get_text().strip() # 구독자수
        view = tr.select_one('.view_cnt').get_text().strip() # 조회수
        video = tr.select_one('.video_cnt').get_text().strip() # 비디오수
        # 리스트에 저장
        lines.append([rank, category, channel, subscriber, view, video])

  0%|          | 0/10 [00:00<?, ?it/s]

In [60]:
# 데이터 프레임으로 변환
df = pd.DataFrame(lines, columns = ['순위', '카테고리', '채널명', '구독자수', '조회수', '비디오수'])

In [61]:
df

,순위,카테고리,채널명,구독자수,조회수,비디오수
0,1,음악/댄스/가수,BLACKPINK,8380만,286억1398만,466개
1,2,음악/댄스/가수,BANGTANTV,7300만,191억4327만,"2,086개"
2,3,음악/댄스/가수,HYBE LABELS,6950만,258억4674만,"1,065개"
3,4,음악/댄스/가수,SMTOWN,3130만,262억1822만,"4,052개"
4,5,키즈/어린이,Boram Tube Vlog [보람튜브 브이로그],2650만,110억5288만,223개
...,...,...,...,...,...,...
995,995,미분류,ASMR Boyoung 반보영,69만,1억2626만,219개
996,998,키즈/어린이,꾸삐KUPI,69만,4억5533만,846개
997,997,IT/기술/컴퓨터,안될과학 Unrealscience,69만,8053만,518개
998,1001,패션/미용,会社員J,69만,1억5546만,533개


In [62]:
# 컬럼에 함수 적용
for column in df.columns[3:]:
    df[column] = df[column].apply(convert_unit)
df

,순위,카테고리,채널명,구독자수,조회수,비디오수
0,1,음악/댄스/가수,BLACKPINK,"83,800,000","28,613,980,000",466
1,2,음악/댄스/가수,BANGTANTV,"73,000,000","19,143,270,000","2,086"
2,3,음악/댄스/가수,HYBE LABELS,"69,500,000","25,846,740,000","1,065"
3,4,음악/댄스/가수,SMTOWN,"31,300,000","26,218,220,000","4,052"
4,5,키즈/어린이,Boram Tube Vlog [보람튜브 브이로그],"26,500,000","11,052,880,000",223
...,...,...,...,...,...,...
995,995,미분류,ASMR Boyoung 반보영,"690,000","126,260,000",219
996,998,키즈/어린이,꾸삐KUPI,"690,000","455,330,000",846
997,997,IT/기술/컴퓨터,안될과학 Unrealscience,"690,000","80,530,000",518
998,1001,패션/미용,会社員J,"690,000","155,460,000",533


In [63]:
# csv 파일로 변환
df.to_csv('youtubeRank.csv')